In [1]:
import math

def calculator(inp: str) -> float:
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

In [2]:
# Make sure you use the OracleResult from the debugging_framework library
from debugging_framework.input.oracle import OracleResult

def oracle(inp: str):
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.FAILING
    
    return OracleResult.PASSING

In [3]:
from debugging_framework.types import Grammar
from debugging_framework.fuzzingbook.grammar import is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
assert is_valid_grammar(CALCGRAMMAR)

Help functions:
TODO

In [4]:
def count_expansions(children):
    if children == []:
        return 0

    counter = 1

    for child in children:
        _, next_children = child
        counter += count_expansions(next_children)    
    
    return counter

In [5]:
def calculate_sophisticated_score_structure(children, height):
    score = 0
    
    for child in children:
        node, next_children = child        
        score += len(next_children)**height
        score += calculate_sophisticated_score_structure(next_children, height+1)

    return score

In [6]:
def calculate_height_and_degreeSum(children, height):
    max_height = height
    score = 0
    
    for child in children:
        node, next_children = child
        score += len(next_children)**height
        next_score, next_height = calculate_height_and_degreeSum(next_children, height+1)
        score += next_score
        if next_height > max_height:
            max_height = next_height
            
    return score, max_height

Fitness functions:
TODO

In [7]:
from evogfuzz.input import Input

In [8]:
def naive_fitness_function(test_input: Input) -> int:
    score_structure = len(str(test_input))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [9]:
def improved_fitness_function(test_input: Input) -> float:
    _, children = test_input.tree
    number_expansions = count_expansions(children)
    lam = 100
    score_structure = (number_expansions**2)/(lam * len(str(test_input)))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [13]:
def sophisticated_fitness_function(test_input: Input) -> float:
    score_structure = calculate_sophisticated_score_structure([test_input.tree],0)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 2**100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [11]:
def ratio_sophisticated_fitness_function(test_input: Input) -> float:
    #_, children = test_input.tree
    lam = 2**50
    degreeSums, height = calculate_height_and_degreeSum([test_input.tree],0)
    score_structure = degreeSums/(lam * height)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

Calculate exception inputs for each fitness function
TODO

In [14]:
from evogfuzz.evogfuzz_class import EvoGFuzz

initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)']

epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10
)

epp_naive = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=naive_fitness_function,
    iterations=10
)

epp_improved = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=improved_fitness_function,
    iterations=10
)

epp_sophisticated = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=sophisticated_fitness_function,
    iterations=10
)

epp_ratio_sophisticated = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=ratio_sophisticated_fitness_function,
    iterations=10
)



found_exception_inputs = epp.fuzz()
found_exception_inputs_naive = epp_naive.fuzz()
found_exception_inputs_improved = epp_improved.fuzz()
found_exception_inputs_sophisticated = epp_sophisticated.fuzz()
found_exception_inputs_ratio_sophisticated = epp_ratio_sophisticated.fuzz()


In [16]:
print("standard:", len(found_exception_inputs))
print("naive:", len(found_exception_inputs_naive))
print("improved:", len(found_exception_inputs_improved))
print("sophisticated:", len(found_exception_inputs_sophisticated))
print("ratio_sophisticated:", len(found_exception_inputs_ratio_sophisticated))


standard: 387
naive: 313
improved: 0
sophisticated: 127
ratio_sophisticated: 0
